In [1]:
# traing data into this folder
!ls /veld/input/

bla.txt


In [2]:
# code here
!ls /veld/executable/

train.ipynb


In [6]:
# save models here
!ls /veld/output/models/m1/

word2vec.model_de	       word2vec.model_de.wv.vectors.npy
word2vec.model_de.syn1neg.npy


In [4]:
# save evaluation here
!ls /veld/output/evaluation/

veld.yaml


In [2]:
import gensim

In [8]:
model = gensim.models.Word2Vec.load("/veld/output/models/m1/word2vec.model_de")

In [9]:
word ="Wien"
print(f'Similaries to {word}:')
for similar_word, similarity in model.wv.most_similar(word, topn=10):
    print(f"- {similar_word}: {similarity}")

Similaries to Wien:
- Graz: 0.8635727763175964
- Innsbruck: 0.8126301765441895
- Linz: 0.8041017651557922
- Salzburg: 0.7798072099685669
- Klagenfurt: 0.7620077729225159
- Budapest: 0.7134349346160889
- Prag: 0.7001412510871887
- Mödling: 0.6942557692527771
- Wiener: 0.6934540867805481
- Gmunden: 0.6655656695365906
